# Library

In [3]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as ds
import torchvision.transforms as transforms

import lightning as L
from lightning.pytorch.trainer import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import LabelEncoder

# Pytorch Practice

In [73]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# seed
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

# hyperparameter
batch_size = 32
epochs = 100
learning_rate = 1e-2
hidden_dim = 32

diamonds = sns.load_dataset('diamonds')

label_encoders = {}
for column in ('cut', 'color', 'clarity'):
    label_encoder = LabelEncoder()
    diamonds.loc[:, column] = label_encoder.fit_transform(diamonds[column])

    label_encoders.update({column: label_encoder})


diamonds = diamonds.drop_duplicates().reset_index(drop=True)

train, temp = train_test_split(diamonds, test_size=0.4, random_state=seed)
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

standard_scaler = StandardScaler()

train.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']] = standard_scaler.fit_transform(train.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']])
valid.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']] = standard_scaler.transform(valid.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']])
test.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']] = standard_scaler.transform(test.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']])

class DiamondsDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = torch.from_numpy(self.data.iloc[idx].drop('cut').values).float() # .values 판다스를 numpy로 바꿔줌
        y = torch.tensor([self.data.iloc[idx].cut]).long() # tensor는 단순 float을 tensor로 바꿔주는데  []안에 넣어야함

        return {
            'X' : X,
            'y' : y,
        }
    

train_dataset = DiamondsDataset(train)
valid_dataset = DiamondsDataset(valid)
test_dataset = DiamondsDataset(test)

train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    drop_last = True,
)

valid_dataloader = DataLoader(
    dataset = valid_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last = True,
)

test_dataloader = DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last = True,
)

class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.linear = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.output(x)

        return x

model = Model(
    input_dim= len(diamonds.columns)-1,
    hidden_dim= hidden_dim,
    output_dim= train['cut'].nunique(),
).to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(
    model.parameters(),
    lr = learning_rate,
)

train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in tqdm(range(1, epochs+1)):
    ##################### Train ###########################
    total_train_loss = 0
    total_train_acc = 0

    model.train()

    for data in train_dataloader:
        X = data.get('X').to(device)
        y = data.get('y').to(device)
        y = y.squeeze()
        
        optimizer.zero_grad() # 기울기 초기화
        output = model(X)
        logit = F.softmax(output, dim =-1)
        train_loss = criterion(logit, y) 
        train_loss.backward()
        optimizer.step() # 파라미터 업데이트

        train_acc = (logit.argmax(dim=-1) == y).float().mean()
        total_train_loss += train_loss
        total_train_acc += train_acc

    mean_train_loss = total_train_loss /len(train_dataloader)
    mean_train_acc = total_train_acc /len(train_dataloader)
    train_losses.append(mean_train_loss)
    train_accs.append(mean_train_acc)
    
    ##################### Validation #############################

    total_valid_loss = 0
    total_valid_acc = 0

    model.eval()
    with torch.no_grad(): # valid는 기울기를 안써서 없애줌줌
        for data in valid_dataloader:
            X = data.get('X').to(device)
            y = data.get('y').to(device)
            y = y.squeeze() # [32, 1]에서 1을 날려줌 [32]
    
            output = model(X)
            logit = F.softmax(output, dim= -1)
            valid_loss = criterion(logit, y)

            valid_acc = (logit.argmax(dim=-1) == y).float().mean()
            total_valid_loss += valid_loss
            total_valid_acc += valid_acc

    mean_valid_loss = total_valid_loss /len(valid_dataloader)
    mean_valid_acc = total_valid_acc /len(valid_dataloader)
    valid_losses.append(mean_valid_loss)
    valid_accs.append(mean_valid_acc)

    print(f'Epoch: {epoch} | train_loss: {mean_train_loss: .4f} | train_acc: {mean_train_acc*100: .2f}% | valid_loss: {mean_valid_loss: .4f} | valid_acc: {mean_valid_acc*100: .2f}%'  )






C:\Users\USER\AppData\Local\Temp\ipykernel_13916\334355076.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  diamonds.loc[:, column] = label_encoder.fit_transform(diamonds[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_13916\334355076.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  diamonds.loc[:, column] = label_encoder.fit_transform(diamonds[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_13916\334355076.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast to

Epoch: 1 | train_loss:  1.2623 | train_acc:  64.29% | valid_loss:  1.2169 | valid_acc:  68.82%


  2%|▏         | 2/100 [00:25<21:07, 12.94s/it]

Epoch: 2 | train_loss:  1.2174 | train_acc:  68.73% | valid_loss:  1.2083 | valid_acc:  69.70%


  3%|▎         | 3/100 [01:00<36:37, 22.65s/it]

...

Epoch: 54 | train_loss:  1.1437 | train_acc:  75.95% | valid_loss:  1.1537 | valid_acc:  74.82%


 55%|█████▌    | 55/100 [16:14<08:32, 11.38s/it]

Epoch: 55 | train_loss:  1.1468 | train_acc:  75.62% | valid_loss:  1.1462 | valid_acc:  75.61%


 55%|█████▌    | 55/100 [16:21<13:22, 17.84s/it]


KeyboardInterrupt: 

In [88]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# seed
seed = 12
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

# hyperparameter
batch_size = 32
epochs = 100
learning_rate = 1e-2
hidden_dim = 32

diamonds = sns.load_dataset('diamonds')

label_encoders = {}
for column in ('cut', 'color', 'clarity'):
    label_encoder = LabelEncoder()
    diamonds.loc[:, column] = label_encoder.fit_transform(diamonds[column])

    label_encoders.update({column: label_encoder})


diamonds = diamonds.drop_duplicates().reset_index(drop=True)

train, temp = train_test_split(diamonds, test_size=0.4, random_state=seed)
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

standard_scaler = StandardScaler()

train.loc[:, ['carat', 'depth', 'table', 'price','x', 'y', 'z']] = standard_scaler.fit_transform(train.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']])
valid.loc[:, ['carat', 'depth', 'table', 'price','x', 'y', 'z']] = standard_scaler.transform(valid.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']])
test.loc[:, ['carat', 'depth', 'table', 'price','x', 'y', 'z']] = standard_scaler.transform(test.loc[:, ['carat', 'depth', 'table', 'price',	'x', 'y', 'z']])

class DiamondsDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = torch.from_numpy(self.data.iloc[idx].drop('price').values).float() # .values 판다스를 numpy로 바꿔줌
        y = torch.tensor([self.data.iloc[idx].price]).float() # tensor는 단순 float을 tensor로 바꿔주는데  []안에 넣어야함

        return {
            'X' : X,
            'y' : y,
        }
    

train_dataset = DiamondsDataset(train)
valid_dataset = DiamondsDataset(valid)
test_dataset = DiamondsDataset(test)

train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    drop_last = True,
)

valid_dataloader = DataLoader(
    dataset = valid_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last = True,
)

test_dataloader = DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last = True,
)

class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.linear = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.output(x)

        return x

model = Model(
    input_dim= len(diamonds.columns)-1,
    hidden_dim= hidden_dim,
    output_dim= 1,
).to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(
    model.parameters(),
    lr = learning_rate,
)

train_losses = []
#train_accs = []
valid_losses = []
#valid_accs = []

for epoch in tqdm(range(1, epochs+1)):
    ##################### Train ###########################
    total_train_loss = 0
    #total_train_acc = 0

    model.train()

    for data in train_dataloader:
        X = data.get('X').to(device)
        y = data.get('y').to(device)
        #y = y.squeeze()
        
        optimizer.zero_grad() # 기울기 초기화
        output = model(X)
        logit = F.relu(output)
        train_loss = criterion(output, y) 
        train_loss.backward()
        optimizer.step() # 파라미터 업데이트

        #train_acc = (logit.argmax(dim=-1) == y).float().mean()
        total_train_loss += train_loss
        #total_train_acc += train_acc

    mean_train_loss = total_train_loss /len(train_dataloader)
    #mean_train_acc = total_train_acc /len(train_dataloader)
    train_losses.append(mean_train_loss)
    #train_accs.append(mean_train_acc)
    
    ##################### Validation #############################

    total_valid_loss = 0
    #total_valid_acc = 0

    model.eval()
    with torch.no_grad(): # valid는 기울기를 안써서 없애줌줌
        for data in valid_dataloader:
            X = data.get('X').to(device)
            y = data.get('y').to(device)
            #y = y.squeeze() # [32, 1]에서 1을 날려줌 [32]
    
            output = model(X)
            logit = F.relu(output)
            valid_loss = criterion(output, y)

            #valid_acc = (logit.argmax(dim=-1) == y).float().mean()
            total_valid_loss += valid_loss
            #total_valid_acc += valid_acc

    mean_valid_loss = total_valid_loss /len(valid_dataloader)
    #mean_valid_acc = total_valid_acc /len(valid_dataloader)
    valid_losses.append(mean_valid_loss)
    #valid_accs.append(mean_valid_acc)

    print(f'Epoch: {epoch} | train_loss: {mean_train_loss: .4f} | valid_loss: {mean_valid_loss: .4f}'  )






C:\Users\USER\AppData\Local\Temp\ipykernel_13916\4132712427.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  diamonds.loc[:, column] = label_encoder.fit_transform(diamonds[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_13916\4132712427.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  diamonds.loc[:, column] = label_encoder.fit_transform(diamonds[column])
C:\Users\USER\AppData\Local\Temp\ipykernel_13916\4132712427.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast

Epoch: 1 | train_loss:  0.0860 | valid_loss:  0.0774


  2%|▏         | 2/100 [00:21<17:19, 10.61s/it]

Epoch: 2 | train_loss:  0.0745 | valid_loss:  0.0826


  3%|▎         | 3/100 [00:31<17:03, 10.55s/it]

...

Epoch: 41 | train_loss:  0.0476 | valid_loss:  0.1832


 42%|████▏     | 42/100 [11:38<10:46, 11.15s/it]

Epoch: 42 | train_loss:  0.0481 | valid_loss:  0.1778


 42%|████▏     | 42/100 [11:39<16:05, 16.65s/it]


KeyboardInterrupt: 

In [148]:
import numpy as np
from sklearn.metrics import mean_squared_error

# 예측값과 실제값을 넣어 계산합니다.

result = []
model.eval()
with torch.no_grad():
    for data in test_dataloader:
        X = data.get('X').to(device)
        y = data.get('y').to(device)

        output = model(X)
        result.append(output)

In [197]:
y_pred = torch.concat(result).squeeze()
df = pd.DataFrame({
    'y_pred': y_pred,
    'groun_truth': test.price[:-7],
    })

df = df * standard_scaler.scale_[3] + standard_scaler.mean_[3]
df = df.reset_index(drop=True).reset_index()
px.scatter(
    df.sample(n = 1000),
    x = 'index',
    y = ['y_pred','groun_truth']
)

AttributeError: 'DataFrame' object has no attribute 'price'

In [236]:
titanic = sns.load_dataset('titanic')
titanic


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [241]:
pclass	sex	age	sibsp	parch	fare	embarked	class	who	adult_male	deck	embark_town	alive	alone

ValueError: could not convert string to float: 'male'

In [432]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# seed
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

# hyperparameter
batch_size = 32
epochs = 100
learning_rate = 1e-3
hidden1_dim = 32


titanic = sns.load_dataset('titanic')
titanic['class_'] = titanic['class']
titanic = titanic.drop('class', axis=1)
titanic.age = titanic.age.fillna(titanic.age.mean())
titanic.deck = titanic.deck.fillna({'Nan':'N'})
titanic.dropna().reset_index(drop=True)



titanic = titanic.drop_duplicates().reset_index(drop=True)

titanic = titanic.drop(columns=['alive', 'alone'])

train, temp = train_test_split(titanic, test_size=0.4, random_state=seed)
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

standard_scaler = StandardScaler()

train.loc[:, ['age', 'fare']]= standard_scaler.fit_transform(train.loc[:, ['age', 'fare']])
valid.loc[:, ['age', 'fare']]= standard_scaler.transform(valid.iloc[:, 1:].loc[:, ['age', 'fare']])
test.loc[:, ['age', 'fare']]= standard_scaler.transform(test.loc[:, ['age', 'fare']])


label_encoders = {}
for column in ('sex', 'embarked', 'class_', 'who', 'adult_male', 'deck', 'embark_town'):
    label_encoder = LabelEncoder()
    train.loc[:, column] = label_encoder.fit_transform(train[column])

    label_encoders.update({column: label_encoder})

label_encoders = {}
for column in ('sex', 'embarked', 'class_', 'who', 'adult_male', 'deck', 'embark_town'):
    label_encoder = LabelEncoder()
    valid.loc[:, column] = label_encoder.fit_transform(valid[column])

    label_encoders.update({column: label_encoder})

label_encoders = {}
for column in ('sex', 'embarked', 'class_', 'who', 'adult_male', 'deck', 'embark_town'):
    label_encoder = LabelEncoder()
    test.loc[:, column] = label_encoder.fit_transform(test[column])

    label_encoders.update({column: label_encoder})


train = train.astype(np.float32)
valid = valid.astype(np.float32)
test = test.astype(np.float32)




class TitanicDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = torch.from_numpy(self.data.iloc[idx].drop('survived').values).float()# .values 판다스를 numpy로 바꿔줌
        y = torch.tensor([self.data.iloc[idx].survived]).float() # tensor는 단순 float을 tensor로 바꿔주는데  []안에 넣어야함

        return {
            'X' : X,
            'y' : y,
        }

C:\Users\USER\AppData\Local\Temp\ipykernel_13916\33788908.py:47: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 2 2 2 2 1 1 2 2 2 1 2 2 1 2 2 2 2 0 0 0 2 2 0 1 0 1 2 2 0 1 0 0 2 2 2 0
 2 1 2 0 0 2 1 0 1 0 1 2 0 1 2 0 0 2 2 2 0 0 0 0 1 1 0 1 2 2 0 0 1 0 2 2 1
 2 2 0 1 0 0 1 0 0 1 0 1 0 0 2 2 2 0 1 0 0 2 2 1 0 2 2 1 2 0 0 1 0 1 2 2 2
 2 2 1 1 0 1 2 2 2 2 0 2 2 1 2 0 2 0 0 0 1 2 2 0 0 1 1 0 0 1 0 2 0 0 2 2 2
 2 1 2 0 0 1 2 2 2 2 1 2 2 2 2 0 0 2 1 2 2 1 2 2 2 2 0 0 1 0 1 0 2 2 1 2 2
 1 0 2 0 0 2 2 2 1 2 2 1 2 2 2 2 2 2 0 0 0 0 1 0 0 1 2 2 2 2 2 0 2 0 2 1 1
 1 1 2 2 2 2 0 2 1 2 1 2 0 0 2 2 2 2 2 0 2 1 2 2 0 0 0 1 2 1 1 0 2 2 0 1 1
 2 2 0 0 2 2 2 1 0 0 0 2 2 2 2 1 2 1 0 0 2 2 1 2 2 1 0 0 0 1 2 2 0 0 0 2 0
 1 0 0 2 0 2 2 2 1 0 0 2 2 2 2 2 2 2 2 2 2 1 2 0 2 2 1 2 2 2 1 2 2 1 0 2 0
 0 2 2 2 1 2 2 1 1 1 1 2 0 2 0 1 2 2 2 1 2 2 2 2 1 2 2 2 2 2 0 1 2 2 2 2 2
 2 2 0 2 1 1 1 0 2 0 2 1 1 2 2 1 1 1 0 2 0 0 2 2 2 0 2 2 0 0 0 2 0

In [435]:
train_dataset = TitanicDataset(train)
valid_dataset = TitanicDataset(valid)
test_dataset = TitanicDataset(test)

train_dataloader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    drop_last = True,
)

valid_dataloader = DataLoader(
    dataset = valid_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last = True,
)

test_dataloader = DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
    drop_last = True,
)

class Model(nn.Module):
    def __init__(self, input_dim, hidden1_dim,output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.hidden1_dim = hidden1_dim
        self.output_dim = output_dim

        self.linear = nn.Linear(input_dim, hidden1_dim)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden1_dim, output_dim)

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.output(x)

        return x

model = Model(
    input_dim= len(titanic.columns)-1,
    hidden1_dim= hidden1_dim,
    output_dim= 1,
).to(device)

criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(
    model.parameters(),
    lr = learning_rate,
)

In [436]:
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

for epoch in range(1, epochs+1):
    ##################### Train ###########################
    total_train_loss = 0
    total_train_acc = 0

    model.train()

    for data in train_dataloader:
        X = data.get('X').to(device)
        y = data.get('y').to(device)
        # y = y.squeeze()
        
        optimizer.zero_grad() # 기울기 초기화
        output = model(X)
        train_loss = criterion(output, y) 
        train_loss.backward()
        optimizer.step() # 파라미터 업데이트

        train_acc = (output.argmax(dim=-1) == y).float().mean()
        total_train_loss += train_loss
        total_train_acc += train_acc

    mean_train_loss = total_train_loss /len(train_dataloader)
    mean_train_acc = total_train_acc /len(train_dataloader)
    train_losses.append(mean_train_loss)
    train_accs.append(mean_train_acc)
    
    ##################### Validation #############################

    total_valid_loss = 0
    total_valid_acc = 0

    model.eval()
    with torch.no_grad(): # valid는 기울기를 안써서 없애줌줌
        for data in valid_dataloader:
            X = data.get('X').to(device)
            y = data.get('y').to(device)
            #y = y.squeeze() # [32, 1]에서 1을 날려줌 [32]
    
            output = model(X)
            valid_loss = criterion(output, y)

            valid_acc = (output.argmax(dim=-1) == y).float().mean()
            total_valid_loss += valid_loss
            total_valid_acc += valid_acc

        mean_valid_loss = total_valid_loss /len(valid_dataloader)
        mean_valid_acc = total_valid_acc /len(valid_dataloader)
        valid_losses.append(mean_valid_loss)
        valid_accs.append(mean_valid_acc)

        print(f'Epoch: {epoch} | train_loss: {mean_train_loss: .4f} | train_acc: {mean_train_acc*100: .2f}% | valid_loss: {mean_valid_loss: .4f} | valid_acc: {mean_valid_acc*100: .2f}%'  )




Epoch: 1 | train_loss:  0.6883 | train_acc:  59.15% | valid_loss:  0.6622 | valid_acc:  59.38%
Epoch: 2 | train_loss:  0.6515 | train_acc:  58.93% | valid_loss:  0.6512 | valid_acc:  59.38%
Epoch: 3 | train_loss:  0.6377 | train_acc:  57.81% | valid_loss:  0.6426 | valid_acc:  59.38%
Epoch: 4 | train_loss:  0.6183 | train_acc:  58.93% | valid_loss:  0.6302 | valid_acc:  59.38%
Epoch: 5 | train_loss:  0.6037 | train_acc:  58.26% | valid_loss:  0.6177 | valid_acc:  59.38%
Epoch: 6 | train_loss:  0.5856 | train_acc:  59.60% | valid_loss:  0.6040 | valid_acc:  59.38%
Epoch: 7 | train_loss:  0.5753 | train_acc:  58.04% | valid_loss:  0.5913 | valid_acc:  59.38%
Epoch: 8 | train_loss:  0.5604 | train_acc:  58.48% | valid_loss:  0.5785 | valid_acc:  59.38%
Epoch: 9 | train_loss:  0.5472 | train_acc:  59.15% | valid_loss:  0.5644 | valid_acc:  59.38%
Epoch: 10 | train_loss:  0.5380 | train_acc:  59.15% | valid_loss:  0.5531 | valid_acc:  59.38%
Epoch: 11 | train_loss:  0.5239 | train_acc:  59.

In [407]:
train_losses.sort()

In [408]:
train_losses[0]

tensor(0.4192, grad_fn=<DivBackward0>)

In [376]:
y

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.]])

In [192]:
output

tensor([[-0.0455,  0.6506],
        [-0.1160,  0.7964],
        [ 0.1682,  0.5028],
        [ 0.1514,  0.5548],
        [-0.1862,  0.1890],
        [-0.3271,  0.3104],
        [-0.6174,  0.6595],
        [-0.1333,  0.7524],
        [-0.0140,  0.4195],
        [-0.0430,  0.3491],
        [ 0.0371,  0.6218],
        [-0.0084,  0.5185],
        [    nan,     nan],
        [ 0.1127,  0.4422],
        [-0.3351,  0.5617],
        [-0.3079,  0.6554],
        [ 0.3996,  0.4888],
        [-0.0080,  0.5189],
        [ 0.1136,  0.3526],
        [    nan,     nan],
        [-0.4009,  0.7007],
        [-0.0177,  0.5490],
        [ 0.1759,  0.4964],
        [    nan,     nan],
        [-0.2737,  0.2873],
        [ 0.0559,  0.6829],
        [-0.0750,  0.7060],
        [ 0.0861,  0.3349],
        [ 0.0152,  0.0913],
        [ 0.0080,  0.6350],
        [-0.0695,  0.5461],
        [-0.0400,  0.5186]], grad_fn=<AddmmBackward0>)

In [209]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# seed
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

# hyperparameter
batch_size = 32
epochs = 20
learning_rate = 1e-2
hidden_dim = 32

titanic = sns.load_dataset('titanic')
titanic['class_'] = titanic['class']
titanic = titanic.drop('class', axis=1)

label_encoders = {}
for column in ('sex', 'embarked', 'class_', 'who', 'adult_male', 'deck', 'embark_town', 'alive', 'alone'):
    label_encoder = LabelEncoder()
    titanic.loc[:, column] = label_encoder.fit_transform(titanic[column])

    label_encoders.update({column: label_encoder})



titanic.sex = titanic.sex.astype(float)
titanic.embarked = titanic.embarked.astype(float)
titanic.class_ = titanic.class_.astype(float)
titanic.who = titanic.who.astype(float)
titanic.adult_male = titanic.adult_male.astype(float)
titanic.deck = titanic.deck.astype(float)
titanic.embark_town = titanic.embark_town.astype(float)
titanic.alive = titanic.alive.astype(float)
titanic.alone = titanic.alone.astype(float)


titanic = titanic.drop_duplicates().reset_index(drop=True)

train, temp = train_test_split(titanic, test_size=0.4, random_state=seed)
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

standard_scaler = StandardScaler()

train = standard_scaler.fit_transform(train)
valid = standard_scaler.transform(valid)
test = standard_scaler.transform(test)

class TitanicDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = torch.from_numpy(self.data.iloc[idx].drop('survived').values).float() # .values 판다스를 numpy로 바꿔줌
        y = torch.tensor([self.data.iloc[idx].survived]).float() # tensor는 단순 float을 tensor로 바꿔주는데  []안에 넣어야함

        return {
            'X' : X,
            'y' : y,
        }

C:\Users\USER\AppData\Local\Temp\ipykernel_13916\4003335231.py:27: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 2 0 2 2 0 2 2 1 2 0 2 2 2 1 2 1 2 2 1 1 2 0 2 2 2 0 2 2 0 0 2 1 0 0 2
 2 2 2 2 1 2 1 2 2 2 2 2 2 2 2 0 1 0 0 1 2 1 2 2 0 0 2 0 2 1 2 2 2 1 2 1 2
 2 2 2 2 1 2 2 2 2 0 1 2 2 2 0 2 2 2 0 2 2 2 0 0 1 1 2 2 0 2 2 2 2 2 2 2 0
 2 2 2 2 2 2 1 0 2 1 2 1 1 0 2 2 2 2 2 2 2 2 1 1 1 0 0 2 0 2 2 2 2 1 1 2 2
 1 1 1 0 2 2 2 0 2 2 2 2 2 1 2 2 2 2 0 2 0 2 0 2 2 2 0 2 2 0 1 2 2 1 2 1 2
 0 2 0 2 2 1 1 2 1 0 0 2 2 2 1 2 2 2 2 2 2 2 2 2 0 2 1 2 1 2 0 2 1 0 1 2 1
 2 2 0 2 1 2 1 2 0 2 1 2 1 2 1 1 1 1 2 2 1 2 2 0 2 1 0 1 2 2 0 2 2 2 0 0 0
 1 2 2 0 0 2 1 2 2 0 0 0 2 1 0 2 0 2 1 2 2 2 2 2 2 0 2 2 2 1 2 0 0 1 2 2 0
 2 0 0 0 2 2 2 1 2 0 0 0 1 0 0 0 1 2 1 2 1 1 0 0 2 2 1 1 2 0 2 1 2 0 2 0 0
 2 0 2 0 0 2 0 1 0 1 1 1 1 1 2 2 2 2 0 2 2 2 2 0 1 2 2 2 1 2 2 2 2 0 2 2 0
 0 2 2 0 2 0 2 0 2 2 0 2 2 0 2 1 2 1 2 1 0 2 2 0 2 2 2 1 1 1 2 2

# Pytorch Lightning

PyTorch Lightning은 PyTorch의 강력함을 유지하면서도 코드의 재사용성과 관리 편의성을 크게 개선시킨 <br>
주로 복잡한 훈련 루프를 추상화하여 코드의 간결성을 높이고, 디버깅과 스케일링을 용이하게 만드는 것을 주된 목표로 함 <br>

<br>

<span style="font-size: 20pt;"> 장점 </span> 

1. 모듈화된 구조: PyTorch Lightning은 훈련, 검증, 테스트 루프와 옵티마이저, 데이터 로더 등을 각각의 모듈로 분리하여 제공 <br>
&nbsp;&nbsp;&nbsp;&nbsp; -> 훨씬 읽기 쉽고 관리하기 쉽게 만들어 줌 (코드의 간결성 확보)
2. 자동화된 훈련 루프: LightningModule 클래스를 상속받아 모델을 정의할 때, 훈련 루프에서 필요한 많은 부분들(forward, loss, optimizer 등)이 자동으로 처리 <br>
&nbsp;&nbsp;&nbsp;&nbsp; -> 사용자가 코드의 일부를 더 간결하게 작성할 수 있도록 해줌
3. 스케일링과 분산 훈련 지원: multi-GPU나 분산 훈련을 지원하여 더 큰 데이터셋이나 복잡한 모델을 쉽게 처리할 수 있게 해줌
5. 확장성: 기존의 PyTorch 코드를 LightningModule로 변환하는 과정이 비교적 간단하고, 다양한 연구와 프로젝트에 쉽게 적용할 수 있음

## 설치 방법

In [ ]:
!pip install lightning

## 사용 방법

### Hyperparameters

In [2]:
batch_size = 128
epochs = 50
learning_rate = 1e-3
hidden_dim = 64

### LightningDataModule

학습 시 사용할 DataLoader를 호출하는 클래스 <br>
관련된 전처리 등을 내부에서 처리하여 사용 <br>

<br>

<span style="font-size:150%">코드</span>

> ```python
> class CustomDataModule(L.LightningDataModule):
>     def __init__(
>         self,
>         batch_size: int,
>         num_workers: int,
>         ):
>         super().__init__()
>         self.num_workers = 4
>         self.batch_size = batch_size
> 
>     def prepare(self, train, valid, test):     
>         self.train, self.valid, self.test = train, valid, test
> 
>     def setup(self, stage: str):
>         if stage == "fit":      
>             self.train_data = self.train
>             self.valid_data = self.valid
> 
>         if stage == "test":     
>             self.test_data = self.test
> 
>     def train_dataloader(self):
>         return DataLoader(
>             dataset=self.train_data,
>             batch_size=self.batch_size,
>             shuffle=False,
>         )
> 
>     def val_dataloader(self):
>         return DataLoader(
>             dataset=self.val_data,
>             batch_size=self.batch_size,
>             shuffle=False,
>         )
> 
>     def test_dataloader(self):
>         return DataLoader(
>             dataset=self.test_data,
>             batch_size=self.batch_size,
>             shuffle=False,
>         )

In [441]:
class TitanicDataModule(L.LightningDataModule):
    def __init__(
         self,
         batch_size: int,
         ):
         super().__init__()
         self.num_workers = 4
         self.batch_size = batch_size
 
    def prepare(self, train_dataset, valid_dataset, test_dataset):     
        self.train_dataset, self.valid_dataset, self.test_dataset = train_dataset, valid_dataset, test_dataset

    def setup(self, stage: str):
        if stage == "fit":      
            self.train_data = self.train_dataset
            self.valid_data = self.valid_dataset

        if stage == "test":     
            self.test_data = self.test_dataset

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train_data,
            batch_size=self.batch_size,
            shuffle=False,
            drop_last = True,
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.valid_data,
            batch_size=self.batch_size,
            shuffle=False,
            drop_last = True,
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test_data,
            batch_size=self.batch_size,
            shuffle=False,
            drop_last = True,
        )

### LightningModule

학습 과정에 사용될 코드를 모듈화 한 클래스 <br>
self.log()를 통해 logging 사용 가능 

<br>

<span style="font-size:150%">코드</span>

> ```python
> class LightningMLP(L.LightningModule):
>     def __init__(
>         self,
>         model: torch.nn,    # 구축한 모델
>         learning_rate: float,
>         ):
>         super().__init__()
>         self.model = model
>         self.learning_rate = learning_rate
>     
>     def training_step(self, batch, batch_idx):      # train 코드
>         pass
>     
>     def on_train_epoch_end(self, *args, **kwargs):  # train이 1 epoch 끝날 때 실행될 코드, 주로 metric 출력을 위해 사용
>         pass
>     
>     def validation_step(self, *args, **kwargs):     # validation 코드
>         pass
>     
>     def on_validation_epoch_end(self):              # valid가 1 epoch 끝날 때 실행될 코드, 주로 metric 출력을 위해 사용
>         pass
>     
>     def test_step(self, *args, **kwargs):           # test 코드
>         pass
>     
>     def configure_optimizers(self):                 # optimizer 설정
>         pass
> ```

In [442]:
class TitanicModule(L.LightningModule):
    def __init__(
        self,
        model: torch.nn,    # 구축한 모델
        learning_rate: float,
        ):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

        self.total_train_loss = []
        self.total_train_acc = []
        self.total_valid_loss = []
        self.total_valid_acc = []
    
    def training_step(self, batch, batch_idx):      # train 코드
        if batch_idx == 0:
            self.total_train_loss.clear()
            self.total_train_acc.clear()

        X = batch.get('X')
        y = batch.get('y')
        y = y.squeeze()

        output = self.model(X)
        logit = F.softmax(output, dim= -1)
        loss = F.cross_entropy(logit, y)

        predicted_label = logit.argmax(dim= -1)
        acc = (predicted_label == y).float().mean()

        self.total_train_loss.append(loss.item()) 
        self.total_train_acc.append(acc.item()) 


        return loss
    
    def on_train_epoch_end(self, *args, **kwargs):  # train이 1 epoch 끝날 때 실행될 코드, 주로 metric 출력을 위해 사용
        print(f'train_loss: {np.mean(self.total_train_loss): .4f} | train_acc: {np.mean(self.total_train_acc)*100: .2f}%')
    
    def validation_step(self, batch, batch_idx):     # validation 코드
        if batch_idx == 0:
            self.total_valid_loss.clear()
            self.total_valid_acc.clear()

        X = batch.get('X')
        y = batch.get('y')
        y = y.squeeze()

        output = self.model(X)
        logit = F.softmax(output, dim= -1)
        loss = F.cross_entropy(logit, y)

        predicted_label = logit.argmax(dim= -1)
        acc = (predicted_label == y).float().mean()

        self.total_valid_loss.append(loss.item()) 
        self.total_valid_acc.append(acc.item()) 


        return loss
    
    def on_validation_epoch_end(self):              # valid가 1 epoch 끝날 때 실행될 코드, 주로 metric 출력을 위해 사용
        print(f'valid_loss: {np.mean(self.total_train_loss): .4f} | valid_acc: {np.mean(self.total_train_acc)*100: .2f}%')
    
    
    def test_step(self, batch):           # test 코드
        X = batch.get('X')
        y = batch.get('y')
        y = y.squeeze()

        output = self.model(X)
        logit = F.softmax(output, dim= -1)

        predicted_label = logit.argmax(dim= -1)

        return predicted_label
    
    def configure_optimizers(self):                 # optimizer 설정
        optimizer = optim.Adam(
            self.model.parameters(),
            lr= self.learning_rate,
        ) 

        return {'optimizer': optimizer}

In [447]:
titanic_data_module = TitanicDataModule(batch_size)
titanic_data_module.prepare(train_dataset, valid_dataset, test_dataset)

In [448]:
model = Model(len(train.columns)-1, 32, 2)
titanic_module = TitanicModule(model, learning_rate)

In [449]:
trainer = Trainer(
    max_epochs=epoch,   
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [450]:
trainer.fit(
    model = titanic_module,
    datamodule = titanic_data_module,
)


  | Name  | Type  | Params | Mode 
----------------------------------------
0 | model | Model | 482    | train
----------------------------------------
482       Trainable params
0         Non-trainable params
482       Total params
0.002     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\USER\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.



RuntimeError: expected scalar type Long but found Float

### Trainer

학습에 필요한 내용을 정의하는 클래스

<br>

<span style="font-size:150%">코드</span>

> ```python
> trainer = Trainer(
>     # accelerator='gpu',    # gpu 사용 시 
>     # devices=2,            # gpu 사용 시 장치 위치 입력 ex) 0, 1, 2, [0, 1], [0, 3], ...
>     max_epochs=epochs,
>     callbacks=[             # callback
>         EarlyStopping(monitor='val_loss', mode='min', patience=10)], # val_loss를 기준으로 최솟값이 10번 이상 업데이트되지 않으면 학습 중지
>     log_every_n_steps=1,
> )
> ```

<br>

생성된 trainer 객체를 이용하여 학습 진행

> ```python
> trainer.fit(
>     model=custom_model,
>     datamodule=custom_data_module,
> )
> ```

### Test

학습이 진행되면서 checpoint가 생성되는데, 이를 통해 학습된 모델 사용 가능

<br>

<span style="font-size:150%">코드</span>

> ```python
> model = CustomDataModule.load_from_checkpoint(path)
> ```

<br>
<br>
<br>

학습된 모델을 가지고 테스트 데이터에 대해 아래와 같이 사용 가능

<br>

<span style="font-size:150%">코드</span>

> ```python
> predictions = trainer.predict(model: L.LightningModule, data_loader)
> ```